In [49]:
import pandas as pd
import mat73
from datetime import datetime,timedelta
import numpy as np
from tqdm import tqdm
from utils import *
import h5py

In [23]:
df_brandon = pd.read_csv('work_tables/EEGs_And_Reports_20231024.csv')

In [24]:
df = pd.read_excel('tables/batch1.xlsx')
df = df.rename({'file':'HashFolderName'},axis=1)
df = df.rename({'time':'eventTime'},axis=1)
df=df.merge(df_brandon[['HashFolderName','SiteID','EEGFolder']],on='HashFolderName')

# generate event time relative to event start
df['startTime'] = df.HashFolderName.apply(get_time_from_filename)
df['relativeTime'] = ((df.eventTime-df.startTime).dt.total_seconds()).astype(int)

In [25]:
df[['HashFolderName', 'class', 'annotation', 'eventTime', 'startTime', 'relativeTime']].to_excel('tables_with_relative_time/batch1.xlsx',index=False)

In [26]:
# process dataframe
root = 'bdsp/opendata/EEG/data/'
windowsize = 15
desired_channels= ['Fp1','F3','C3','P3','F7','T3','T5','O1','Fz','Cz','Pz','Fp2','F4','C4','P4','F8','T4','T6','O2']
hf = h5py.File('SauronData.h5', 'w')

In [36]:
# process EEG file
for HashFolderName in tqdm(df.HashFolderName.unique()[10:20]):
    # get all events from same hashfolder
    sub_df = df[df.HashFolderName==HashFolderName]
    sub_df = sub_df.sort_values(by='relativeTime')

    # get knowledge about file location
    SiteID,EEGFolder,HashFolderName = sub_df.iloc[0][['SiteID','EEGFolder','HashFolderName']].to_list()
    # load file and metadata
    data,Fs,channels = process_EEG_file(root,SiteID,EEGFolder,HashFolderName,index=0)
    
    # catch some potential problems
    if not 200 in Fs:
        print(f'frequency is {Fs}, instead of the normal 200!')
        break   
    if not set(desired_channels).issubset(channels):
        print(f'not all desired channels are present!\n want: {desired_channels}\n got: {channels}')
        break

    time_offset = 0
    # save all events from the eeg file
    for j in range(len(sub_df)):
        # get time relative to beginning of file 
        relative_event_time = sub_df.iloc[j].relativeTime - time_offset
        # load next file if relative event time > 12h, since engineers broke up the files!
        if relative_event_time>12*3600:
            time_offset = 12*3600
            data,Fs,channels = process_EEG_file(root,SiteID,EEGFolder,HashFolderName,index=1)

        # get snippet from eeg file and save it somewhere
        snippet = get_event_snippet(data=data,
                                    relative_event_time=relative_event_time,
                                    windowsize=windowsize,
                                    all_channels=channels,
                                    desired_channels=desired_channels,
                                    Fs=Fs)
        name = HashFolderName+'_'+str(relative_event_time)+'.npy'
        # save in h5py file
        # hf.create_dataset(name=name,data=snippet,dtype='f4',compression='gzip')
        np.save('test/'+name,snippet)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:12<01:49, 12.19s/it]


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'bdsp/opendata/EEG/data/S0001/cEEG/0003ecbceff1851d1c25d05d6bb02551fa1bf0b73bd040da9fd0c4686b057459_20190320_110917/0003ecbceff1851d1c25d05d6bb02551fa1bf0b73bd040da9fd0c4686b057459_1_20190320_110917.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [52]:
(sub_df.iloc[0].startTime+timedelta(hours=12)).strftime("%m%d%Y_%H%M%S")

'03202019_230917'

In [ ]:
0003ecbceff1851d1c25d05d6bb02551fa1bf0b73bd040da9fd0c4686b057459_1_20190320_110917
0003ecbceff1851d1c25d05d6bb02551fa1bf0b73bd040da9fd0c4686b057459_1_20190320_230917

In [ ]:
110917